In [1]:
import biutils
import pandas as pd
import pingouin as pg

In [2]:
# retrieve table from database
binary = biutils.get_tbl('binary')
no_out = biutils.get_tbl('bino_out')
zdata = biutils.get_tbl("bizdata")

In [3]:
mcolumns = list(binary.columns)
mcolumns.remove('voxel')
mcolumns.remove('label')
mcolumns.remove('dataset')
metlist = list(set([i.split(" ")[0] for i in mcolumns]))
len(metlist)
metrics = ['FAT14ratio','ratioWAT47_1d','ratioFAT14_1d','ratioFAT54_1d']
oned = ['nws_ffrac','nws_wfrac','nws_wfratio']


In [4]:

intervars =['triglycerylfatcrossp(TGFRupper) FAT14ratio', 
'methylfat(FMETD10) FAT14ratio', 
'olefinicfat(UFD54) FAT14ratio', 
'fat(4p2_3p9) FAT14ratio', 
'methylenefat(FAT21) FAT14ratio', 
'fat(2p06_1p32) ratioWAT47_1d', 
'adc', 'gly+myo FAT14ratio', 
'fat(1p32_0p9) FAT14ratio']

intername = ['TGFRupper/FAT14','FMETD/FAT14','UFD54/FAT14','CP8/FAT14',
'FAT21/FAT14','CP4/FAT14','ADC','mI+Gly/FAT14','CP2/FAT14']

renamed = no_out[intervars+['voxel','dataset','label']]
renamed = renamed.rename(columns=dict(zip(intervars,intername)))


## Statistical tests

### Assumption checks  
Check for equal variances using leneve's test

#### Homegeneity of variance assumptions   
If the data is normally didstributed, we could perform Bartlett's Test for homogeneity of variances.  
However, in this case, this condition does not hold strictly, so we use Levene's Test instead.   ( Levene's test is an inferential statistic used to assess the equality of variances for a variable calculated for two or more groups.)
```python
def group_lenv(dat,metlist):
    """
    perform leneve tests among different label groups 
    with given data and metabolite lists
    """
```


In [6]:
lenv = biutils.group_lenv(no_out,mcolumns)

In [8]:
lenv.head(3)

,W,pval,equal_var
methylfat(FMETD10) ratioFAT54_1d,2.636946,0.105130,True
methylenefat(FAT14) ratioFAT54_1d,12.233011,0.000518,False
methylenefat(FAT21) ratioFAT54_1d,8.064747,0.004726,False


## Two-sided t-tests between maglinant and benign corrected for unequal variances
https://pingouin-stats.org/build/html/generated/pingouin.mwu.html 

In [9]:
tt = pd.DataFrame()
for i in range(len(mcolumns)): # specify variables
    var = mcolumns[i]
    ev = lenv.equal_var[i] # perform tests based on levene result
    bn = no_out.loc[no_out['label']=='benign',var]
    ml = no_out.loc[no_out['label']=='malignant',var] # change no_out to name of datasets
    update = pg.ttest(bn,ml, paired=False,confidence = 0.95, alternative='two-sided', correction=ev)  
    tt = pd.concat([tt,update])
tt['met'] = mcolumns #rename
tt['valid'] = tt['p-val'] < 0.01 #set p-values
varstt = list(tt[tt.valid].met)
print(len(tt[tt.valid].met.unique()))
print("metabolite failing Welch's t-tests")
tt[~tt.valid].met


/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)
/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)
/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/Users/linlin/Li

39
metabolite failing Welch's t-tests


/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)
/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)
/Users/linlin/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/Users/linlin/Li

T-test                  methylfat(FMETD10) ratioFAT54_1d
T-test                 methylenefat(FAT29) ratioFAT54_1d
T-test                        fat39(fat39) ratioFAT54_1d
T-test    methyleneglycerolbackbone(MGB42) ratioFAT54_1d
T-test                  olefinicfat(UFD54) ratioFAT54_1d
T-test       unsatfatrightcrossp(UFRlower) ratioFAT54_1d
T-test        unsatfatleftcrossp(UFLlower) ratioFAT54_1d
T-test       unsatfatrightcrossp(UFRupper) ratioFAT54_1d
T-test        unsatfatleftcrossp(UFLupper) ratioFAT54_1d
T-test     triglycerylfatcrossp(TGFRlower) ratioFAT54_1d
T-test     triglycerylfatcrossp(TGFRupper) ratioFAT54_1d
T-test                       fat(0p9_1p32) ratioFAT54_1d
T-test                      fat(1p64_2p36) ratioFAT54_1d
T-test                      fat(2p36_1p64) ratioFAT54_1d
T-test                        fat(3p9_4p3) ratioFAT54_1d
T-test                             gly+myo ratioFAT54_1d
T-test                     olefinicfat(UFD54) FAT14ratio
T-test           unsatfatleftcr

## Mann-Whitney U

In [10]:
mwu = pd.DataFrame()
for i in range(len(mcolumns)):
    var = mcolumns[i]
    df = no_out.loc[:,[var,'label']]
    update = pg.mwu(x = df[df.label == 'benign'][var],y = df[df.label == 'malignant'][var]) 
    mwu = pd.concat([mwu,update])
mwu['met'] = mcolumns
mwu['valid'] = mwu['p-val'] < 0.01
varsmwu = list(mwu[mwu.valid].met)
print(len(mwu[mwu.valid].met.unique()))


86


In [11]:
print("metabolite failing MWU's tests")
mwu[~mwu.valid].met

metabolite failing MWU's tests


MWU                 tcholine(CHO32) ratioFAT54_1d
MWU                    fat39(fat39) ratioFAT54_1d
MWU                    fat(4p2_3p9) ratioFAT54_1d
MWU                         gly+myo ratioFAT54_1d
MWU       unsatfatleftcrossp(UFLlower) FAT14ratio
MWU      unsatfatrightcrossp(UFRupper) FAT14ratio
MWU    triglycerylfatcrossp(TGFRlower) FAT14ratio
MWU                      fat(0p9_1p32) FAT14ratio
MWU                       fat(3p9_4p3) FAT14ratio
MWU                 tcholine(CHO32) ratioFAT14_1d
MWU                         gly+myo ratioFAT14_1d
MWU                    fat39(fat39) ratioWAT47_1d
MWU                    fat(4p2_3p9) ratioWAT47_1d
Name: met, dtype: object

MWU for selected variables

In [12]:
mwue = pd.DataFrame()
for i in range(len(intervars)): #specify the list of variables
    var = intervars[i]
    df = no_out.loc[:,[var,'label']] #change no_out to the name of the dataset
    update = pg.mwu(x = df[df.label == 'benign'][var],y = df[df.label == 'malignant'][var]) 
    mwue = pd.concat([mwue,update])
mwue['met'] = intername # renaming
mwue['valid'] = mwue['p-val'] < 0.01 # set p-values
mwue.sort_values(['p-val'])  # sort by p_values
# if you want to copy something to clipboard, use .to_clipboard() right after the code


,U-val,alternative,p-val,RBC,CLES,met,valid
MWU,39975.0,two-sided,2.127261e-36,-0.701245,0.850622,ADC,True
MWU,39017.0,two-sided,1.846460e-32,-0.660475,0.830237,FAT21/FAT14,True
MWU,37819.0,two-sided,6.847265e-28,-0.609490,0.804745,CP8/FAT14,True
MWU,36920.0,two-sided,1.068635e-24,-0.571231,0.785615,FMETD/FAT14,True
MWU,12336.0,two-sided,1.444175e-17,0.475008,0.262496,CP2/FAT14,True
MWU,14096.5,two-sided,6.681492e-13,0.400085,0.299957,CP4/FAT14,True
MWU,29902.0,two-sided,1.801668e-07,-0.272561,0.636280,mI+Gly/FAT14,True
MWU,30142.5,two-sided,3.792312e-07,-0.282796,0.641398,UFD54/FAT14,True
MWU,28908.5,two-sided,3.532889e-05,-0.230280,0.615140,TGFRupper/FAT14,True


In [13]:
# variables that fail the mann-whitney u's 
mwu[~mwu.valid][['p-val','met']]

,p-val,met
MWU,0.047315,tcholine(CHO32) ratioFAT54_1d
MWU,0.076737,fat39(fat39) ratioFAT54_1d
MWU,0.037560,fat(4p2_3p9) ratioFAT54_1d
MWU,0.445204,gly+myo ratioFAT54_1d
MWU,0.678907,unsatfatleftcrossp(UFLlower) FAT14ratio
MWU,0.010735,unsatfatrightcrossp(UFRupper) FAT14ratio
MWU,0.229895,triglycerylfatcrossp(TGFRlower) FAT14ratio
MWU,0.882110,fat(0p9_1p32) FAT14ratio
MWU,0.667208,fat(3p9_4p3) FAT14ratio
MWU,0.603634,tcholine(CHO32) ratioFAT14_1d
